<h1>Homework 4 - Zachary Jernigan</h1>

<h3>Loading data and implementing packages</h3>

In [3]:
import pandas as pd
import json

In [11]:
with open('universities.json','r') as f:
    raw_data = json.load(f)

df_main = pd.DataFrame(raw_data)

df_main.head()

,college,content,nrefs,nlinks
0,Air Force Institute of Technology,The Air Force Institute of Technology (AFIT) i...,142,253
1,Albert Einstein College of Medicine,The Albert Einstein College of Medicine is a p...,88,234
2,American University,The American University (AU or American) is a ...,148,423
3,Arizona State University,Arizona State University (Arizona State or ASU...,515,880
4,Arkansas State University,Arkansas State University (A-State or ASU) is ...,109,281


<h3>Creating Sub Groups of R1 and R2 universities</h3>

In [20]:
#this will just look to see if the wiki entry lists them as R1 or R2 and attach that classification to the dataframe
def classify_r(text):
    if "R1" in text:
        return "R1"
    elif "R2" in text:
        return "R2"
    else:
        return "missing"

df_main['Classification'] = df_main['content'].apply(classify_r)

df_main.head()
    

,college,content,nrefs,nlinks,Classification
0,Air Force Institute of Technology,The Air Force Institute of Technology (AFIT) i...,142,253,R2
1,Albert Einstein College of Medicine,The Albert Einstein College of Medicine is a p...,88,234,missing
2,American University,The American University (AU or American) is a ...,148,423,R2
3,Arizona State University,Arizona State University (Arizona State or ASU...,515,880,R1
4,Arkansas State University,Arkansas State University (A-State or ASU) is ...,109,281,R2


In [21]:
#some listed as missing. investigating
df_main['Classification'].value_counts()

Classification
R1         127
R2         118
missing     41
Name: count, dtype: int64

In [ ]:
#Noted the 41 missing. Will investigate them further later.
#For now continuing with df setup

In [8]:
df_r1 = df_main[df_main['Classification']=="R1"]
df_r2 = df_main[df_main['Classification']=="R2"]

df_r1.head()

,college,content,nrefs,nlinks,Classification
3,Arizona State University,Arizona State University (Arizona State or ASU...,515,880,R1
5,Auburn University,Auburn University (AU or Auburn) is a public l...,178,475,R1
10,Baylor University,Baylor University is a private Baptist Christi...,193,669,R1
11,Binghamton University,The State University of New York at Binghamton...,149,380,R1
14,Boston University,Boston University (BU) is a private research u...,467,935,R1


In [9]:
df_r2.head()

,college,content,nrefs,nlinks,Classification
0,Air Force Institute of Technology,The Air Force Institute of Technology (AFIT) i...,142,253,R2
2,American University,The American University (AU or American) is a ...,148,423,R2
4,Arkansas State University,Arkansas State University (A-State or ASU) is ...,109,281,R2
7,Azusa Pacific University,Azusa Pacific University (APU) is a private ev...,86,394,R2
8,Ball State University,"Ball State University (Ball State, State or BS...",164,407,R2


In [22]:
#Wanted to see which schools were misisng classification so I can update them
df_missing = df_main[df_main['Classification']=="missing"]

df_missing['college']


1                    Albert Einstein College of Medicine
6                                     Augusta University
9                             Baylor College of Medicine
13                                        Boston College
21                 California State University, East Bay
29           Center for Measuring University Performance
43                                     Dartmouth College
50                       Eastern Virginia Medical School
65               Icahn School of Medicine at Mount Sinai
70                    Indiana University of Pennsylvania
74                              Johns Hopkins University
79                                 Loma Linda University
82                           Loyola Marymount University
87           Mayo Clinic College of Medicine and Science
88                  Medical University of South Carolina
106                                  Notre Dame, Indiana
112                   Oregon Health & Science University
121                            

In [25]:
#Using a publically available CSV to assist with classification of missing
global df_carnegie 
df_carnegie = pd.read_csv('carnegie_class.csv')
df_carnegie.head()

,UNITID,INSTNM,City,State,2021 Carnegie Classification,HERD FY21,HERD FY22,HERD FY23,HERD FY21-23 AVG,Research Doctorates 2020-21,Research Doctorates 2021-22,Research Doctorates 2022-23,Research Doctorates AVG,2025 Research Activity Designation
0,222178,Abilene Christian University,Abilene,TX,Doctoral/Professional Universities,"$4,597,000","$6,310,000","$8,182,000","$6,363,000",95,49,61,68,Research 2: High Spending and Doctorate Produc...
1,200697,Air Force Institute of Technology-Graduate Sch...,Wright-Patterson AFB,OH,Doctoral Universities: High Research Activity,"$45,703,000","$44,391,000","$56,848,000","$48,980,667",26,45,38,36,Research 2: High Spending and Doctorate Produc...
2,100654,Alabama A & M University,Normal,AL,Master's Colleges & Universities: Larger Programs,"$9,180,000","$9,611,000","$15,588,000","$11,459,667",7,9,3,6,Research Colleges and Universities
3,100724,Alabama State University,Montgomery,AL,Doctoral/Professional Universities,"$2,631,000","$3,178,000","$3,229,000","$3,012,667",14,2,4,7,Research Colleges and Universities
4,188526,Albany College of Pharmacy and Health Sciences,Albany,NY,Special Focus Four-Year: Other Health Professi...,"$3,687,000","$2,276,000","$2,594,000","$2,852,333",0,0,0,0,Research Colleges and Universities


In [26]:
def classification_complex(college):
    found_match = df_carnegie[df_carnegie['INSTNM']==college]

    #if I found a row that matches the college
    if not found_match.empty:
        designation = found_match.iloc[0]['2025 Research Activity Designation']
        if "Research 1" in designation:
            return "R1"
        elif "Research 2" in designation:
            return "R2"

    return "Other"


df_missing['Classification'] = df_missing['college'].apply(classification_complex)

df_missing.head()

/tmp/ipykernel_512/2611015204.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing['Classification'] = df_missing['college'].apply(classification_complex)


,college,content,nrefs,nlinks,Classification
1,Albert Einstein College of Medicine,The Albert Einstein College of Medicine is a p...,88,234,R2
6,Augusta University,Augusta University (AU) is a public research u...,55,281,R2
9,Baylor College of Medicine,Baylor College of Medicine (BCM) is a medical ...,80,360,R1
13,Boston College,Boston College (BC) is a private Jesuit resear...,255,693,R1
21,"California State University, East Bay","California State University, East Bay (Cal Sta...",139,439,Other


In [27]:
#now that the data is a little cleaner can update df_main and reupdate r1 and r2
df_main.update(df_missing)

df_main.head()

,college,content,nrefs,nlinks,Classification
0,Air Force Institute of Technology,The Air Force Institute of Technology (AFIT) i...,142,253,R2
1,Albert Einstein College of Medicine,The Albert Einstein College of Medicine is a p...,88,234,R2
2,American University,The American University (AU or American) is a ...,148,423,R2
3,Arizona State University,Arizona State University (Arizona State or ASU...,515,880,R1
4,Arkansas State University,Arkansas State University (A-State or ASU) is ...,109,281,R2


In [28]:
df_main['Classification'].value_counts()

Classification
R1       137
R2       131
Other     18
Name: count, dtype: int64

In [29]:
#Ok only 18 are now listed as "Other" these are univerisites officially listed as not R1 or R2 by this index

<h3>Starting Analysis</h3>